# Frailty 모델을 적용한 생존 분석 코드: ID별 이질성 고려

* Frailty 모델은 개체 간의 미관측 이질성을 설명하기 위한 효과적인 방법
* 특히, ID별로 생존 시간에 차이가 있을 수 있는 경우, Frailty 모델을 사용하여 이러한 이질성을 고려할 수 있다.

In [9]:
import pandas as pd
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
import numpy as np
import matplotlib.pyplot as plt
from lifelines import *
from matplotlib import rc
plt.rc('font', family='Malgun Gothic')

# 데이터 불러오기
file_path = 'C:/Users/user/Desktop/Survival_Data/Disease/colon.csv'
df = pd.read_csv(file_path, index_col=0)


In [10]:
df['gu'].unique()

array(['금정구', '서구', '영도구', '사상구', '해운대구', '동래구', '사하구', '남구', '부산진구',
       '동구', '강서구', '연제구', '북구', '중구', '수영구', '기장군'], dtype=object)

In [11]:
# 매핑 딕셔너리
gu_mapping = {
    '금정구': 1,
    '서구': 2,
    '영도구': 3,
    '사상구': 4,
    '해운대구': 5,
    '동래구': 6,
    '사하구': 7,
    '남구': 8,
    '부산진구': 9,
    '동구': 10,
    '강서구': 11,
    '연제구': 12,
    '북구': 13,
    '중구': 14,
    '수영구': 15,
    '기장군': 16
}

df['gu'] = df['gu'].map(gu_mapping)

In [12]:
df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,996,2,66,C187,1,4.0,1,64,1,1,0,0,0,1
1,1441,2,46,C180,2,1.0,0,298,1,1,0,0,0,0
2,1658,1,64,C184,3,1.0,1,241,1,1,0,0,0,0
3,2428,1,47,C20,4,2.0,1,104,1,1,0,0,0,1
4,3078,2,57,C20,5,1.0,0,296,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33971,364369,1,82,C20,6,2.0,1,2,1,0,0,0,0,1
33972,364371,1,64,C180,5,4.0,0,25,1,0,0,0,0,1
33973,364376,1,52,C185,8,1.0,0,25,1,0,0,0,0,0
33974,364386,1,56,C182,5,4.0,0,25,1,1,0,0,0,1


In [18]:
# CoxPHFitter를 이용하여 Frailty 모델 적합
data = df[['gu','stime', 'event_inc','tx_1', 'tx_2','tx_3', 'seer_TF','sex']]
cph = CoxPHFitter()
cph.fit(data, duration_col='stime', event_col='event_inc', strata='gu')

# 모델 요약
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 33976 total observations, 19331 right-censored observations>
             duration col = 'stime'
                event col = 'event_inc'
                   strata = gu
      baseline estimation = breslow
   number of observations = 33976
number of events observed = 14645
   partial log-likelihood = -103352.23
         time fit was run = 2024-12-10 04:23:15 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
tx_1      -1.32      0.27      0.02           -1.36           -1.29                0.26                0.28
tx_2      -0.13      0.88      0.02           -0.17           -0.09                0.85                0.91
tx_3      -0.44      0.64      0.03           -0.50           -0.38                0.61                0.69
seer_TF    1.10      3.01      0.02            1.06            1.14                2.89                3.14
sex       -0.10      0.90      0.02           -0.14           -0.07                0.87                0.93

           cmp to      z      p  -log2(p)
covariate                                
tx_1         0.00 -73.35 <0.005       inf
tx_2         0.00  -6.99 <0.005     38.45
tx_3         0.00 -13.97 <0.005    144.98
seer_TF      0.00  52.58 <0.005       inf
sex          0.00  -6.09 <0.005     29.74
---
Concordance = 0.70
Partial AIC = 206714.47
log-likelihood ratio test = 7274.56 on 5 df
-log2(p) of ll-ratio test = inf

In [17]:
# 개별 개체의 Frailty 값 추출 (선택 사항)
frailty_terms = cph.baseline_hazard_
frailty_terms.iloc[:,]

0.0      0.005396
1.0      0.011991
2.0      0.010965
3.0      0.013514
4.0      0.010693
           ...   
272.0    0.000000
276.0    0.000000
288.0    0.000000
296.0    0.000000
298.0    0.000000
Name: 2, Length: 272, dtype: float64

In [19]:
df.to_csv('C:/Users/user/Desktop/colon_gu.csv')

## "구" 데이터들을 더미변수화하고, int 타입

In [25]:
dummy_columns = [col for col in dummy_df.columns if 'gu_' in col] 
dummy_df[dummy_columns]= dummy_df[dummy_columns].astype(int)

In [26]:
dummy_df

,SERIAL_ID,sex,age,icd_10,seercode,event_inc,stime,tx_1,tx_2,tx_3,...,gu_7,gu_8,gu_9,gu_10,gu_11,gu_12,gu_13,gu_14,gu_15,gu_16
0,996,2,66,C187,4.0,1,64,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1441,2,46,C180,1.0,0,298,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1658,1,64,C184,1.0,1,241,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2428,1,47,C20,2.0,1,104,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,3078,2,57,C20,1.0,0,296,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33971,364369,1,82,C20,2.0,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33972,364371,1,64,C180,4.0,0,25,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33973,364376,1,52,C185,1.0,0,25,1,0,0,...,0,1,0,0,0,0,0,0,0,0
33974,364386,1,56,C182,4.0,0,25,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### 코드 해석
* 데이터 불러오기: your_data.csv 파일에서 데이터를 불러옵니다. T는 생존 시간, E는 사건 발생 여부, id는 개체 고유 ID를 나타냅니다.
* CoxPHFitter를 이용한 모델 적합: CoxPHFitter 클래스를 사용하여 Cox 비례 위험 모델을 적합합니다. strata='id' 옵션을 통해 ID를 strata로 설정하여 개체별 Frailty를 추정합니다.
* 모델 요약: print_summary() 함수를 통해 모델의 요약 정보를 출력합니다.
* 개별 개체의 Frailty 값 추출: baseline_hazard_.iloc[-1]을 통해 개체별 Frailty 값을 추출할 수 있습니다. Frailty 값이 클수록 해당 개체의 사망 위험이 높다는 것을 의미합니다.
### Frailty 모델의 의미
* Frailty: 개체 간의 미관측 이질성을 나타내는 숨겨진 변수입니다. 즉, 같은 조건에 놓여 있어도 개체마다 생존 시간이 다르게 나타나는 이유를 설명해줍니다.
* Strata: ID를 strata로 설정함으로써 각 ID별로 다른 baseline hazard를 가질 수 있도록 합니다. 즉, 각 ID마다 고유한 생존 패턴을 가질 수 있다는 것을 의미합니다.

### 추가 고려 사항
* 다른 공변량: cph.fit() 함수에 다른 공변량을 추가하여 분석할 수 있습니다.
* 시간 의존적 공변량: 시간에 따라 변하는 공변량을 고려할 수 있습니다.
* 상호작용 효과: 공변량과 Frailty 간의 상호작용 효과를 분석할 수 있습니다.
* 모델 검증: AIC, BIC 등을 이용하여 모델을 비교하고, 적절한 모델을 선택할 수 있습니다.
* 그래프 시각화: 생존 곡선, 위험 함수 등을 그려 모델 결과를 시각화할 수 있습니다.

### 주의 사항
* 데이터: 데이터의 질이 분석 결과에 큰 영향을 미칩니다. 결측치 처리, 이상치 확인 등의 전처리 과정이 필요합니다.
* 모델 가정: Cox 비례 위험 모델의 가정이 만족되는지 확인해야 합니다.
* 해석: Frailty 값의 해석에 주의해야 합니다. Frailty 값이 높다고 해서 반드시 질병이 더 심각하다는 것을 의미하지는 않습니다.

### 결론
* Frailty 모델은 개체 간의 이질성을 고려하여 더욱 정확한 생존 분석을 가능하게 합니다. 특히, ID별로 생존 시간에 차이가 있을 수 있는 경우, * Frailty 모델을 사용하여 이러한 이질성을 설명할 수 있습니다.